# Test Frame Sender

This file is used for sending individual frames to the device. It measures the accuracy based on individual frames.

**Note that the software on the MCU must match this, and not classify based on clusters of frames.**

In [2]:
import numpy as np
import pandas as pd
import gc

In [4]:
df = pd.read_csv("../Model/dataset.csv", header=None)

In [5]:
dataset = df.iloc[:, :-1].to_numpy(dtype=np.float32)            # All but last column as float32
labels_set = df.iloc[:, -1].to_numpy(dtype=str)                 # Last column as string
del df
gc.collect()

2049

In [13]:
dataset = dataset.reshape(dataset.shape[0], 16, 8, 1)
input_shape = dataset[0].shape

print(f"Dataset shape: {dataset.shape}")
print(f"Labels shape: {labels_set.shape}")
print(f"Input shape: {input_shape}")

Dataset shape: (818512, 16, 8, 1)
Labels shape: (818512,)
Input shape: (16, 8, 1)


In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset, labels_set, test_size=0.05, random_state=42, stratify=labels_set)
del dataset, labels_set
gc.collect()

509

In [15]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Hot end code the labels.
label_encoder = LabelEncoder()
y_train = to_categorical(label_encoder.fit_transform(y_train))
y_test = to_categorical(label_encoder.fit_transform(y_test))

In [7]:
def indexOfMax(arr):
    return np.argmax(arr)

In [37]:
import serial
print("Configuring serial port...")
ser = serial.Serial(
    port='/dev/ttyACM0',  # Change this to your actual port, e.g., 'COM3' on Windows, '/dev/ttyS0' on Linux
    baudrate=115200,       # Set baud rate to 115200
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=1              # Set timeout for reading
)

Configuring serial port...


In [38]:
if ser.is_open:
    print(f"Serial port {ser.port} opened at {ser.baudrate} baud.")

Serial port /dev/ttyACM0 opened at 115200 baud.


In [39]:
def calc_accuracy(y_true, y_pred):
    success = 0
    failure = 0
    for i in range(len(y_pred)):
        if int(y_pred[i]) == int(indexOfMax(y_true[i])):
            success += 1
        else:
            failure += 1
    res = round(success / (success + failure), 4)
    return res

In [40]:
import struct

x = x_test
y = y_test[0:len(x)]
print(x.shape)

results = []
total = len(x)

# Send data
for i in range(total):
    data = x[i].flatten()
    for j in range(len(data)):
        data_to_send = struct.pack('f', data[j])
        ser.write(data_to_send)
        response = ser.readline().decode('utf-8').strip()
        print(f"Sent: {data_to_send} - Received: {response}")
    # No need for sleep here as the delay is already managed by the serial communication timeout.
    # Read response
    #print(f"Guess: {response} - Class: {indexOfMax(y[i])} Accuracy: {calc_accuracy(y, results)} ({i}/{total})")


(40926, 16, 8, 1)
Sent: b'\xc35\xed\xc2' - Received: -118.605003,
Sent: b'7I\xe7\xc2' - Received: -115.642998,
Sent: b'\xe7\xbb\xe6\xc2' - Received: -115.366997,
Sent: b'\x0e\xad\xe0\xc2' - Received: -112.337997,
Sent: b's(\xe5\xc2' - Received: -114.579002,
Sent: b'\xac\x9c\xf1\xc2' - Received: -120.806000,
Sent: b'=\x8a\xf6\xc2' - Received: -123.269997,
Sent: b'F\xf6\xf0\xc2' - Received: -120.481003,


KeyboardInterrupt: 

In [41]:
# Close the serial port
ser.close()